# Google Spreadsheet Interface

## Open Data Science Initiative

### 14th August 2014 Neil Lawrence

### Updated 17th October 2014 Neil Lawrence
### Updated 19th October 2015 Neil Lawrence
### Updated 27th May 2021 Neil Lawrence
### Updated 22nd December 2021 Neil Lawrence

As part of the distribution of information to area chairs in NIPS 2014 we used Google docs to provide summary spreadsheets. Review information is processed in `pandas` so this rough interface is designed to allow uploading and downloading of information from google spreadsheets as pandas data frames.

For this file to work, there needs to be a configuration file called `_googoal.yml` in this subdirectory that specifies the following:

```
google: # Set the email address of an account to access google doc information.
  oauth2_keyfile: # Location of the auth key json file.
  analytics_table: # analytics table number
google_docs:
  user: # User of the google docs accounts
  password: # Set the password of an account to access google doc information.
gmail:
  user: user1@gmail.com
  name: Firstname Lastname
```


First you need to create a set of OAuth client ID credentials. You can do this as a desktop app. Instructions are here: <https://developers.google.com/workspace/guides/create-credentials>

1. Open the [Google Cloud Console](https://console.cloud.google.com/).
2. At the top-left, click Menu > APIs & Services > Credentials.
3. Click Create Credentials > OAuth client ID.
4. Click Application type > Desktop app.
5. In the "Name" field, type a name for the credential. This name is only shown in the Cloud Console.
6. Click Create. The OAuth client created screen appears, showing your new Client ID and Client secret. Download the associated `json` file and save it, for example as `googoal-client-credentials.json`
7. Click OK. The newly created credential appears under "OAuth 2.0 Client IDs."
8. Open the file `_googoal.yml` and add 
```
google: # Set the email address of an account to access google doc information.
  oauth2_keyfile: googoal-client-credentials.json
```
as the `oauth2_keyfile`.

In [1]:
import googoal as gl
import pandas as pd

Next you'll need to make use of two gmail user IDs for this example.


In [2]:
user1 = "nips2014@gmail.com"
user2 = "n.lawrence@sheffield.ac.uk"

First let's create a simple data frame for placing in a spreadsheet.

In [3]:
d = {'one' : pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
    'two' : pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])} 

df = pd.DataFrame(d)

In [4]:
df

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


### Creating a Google Sheet
Now we create the google document and write our data frame, `df`, to the spreadsheet.

In [5]:
sheet = gl.sheet.Sheet()

The spreadsheet is uniquely defined by its resource `id`, which can be recovered from the spreadsheet as follows:

In [6]:
print(sheet.resource._id)

1lv9yYAjhtLp9YmRSPGPx3CaNT9ck4DrqgNuC6CZhMoo


The spreadsheet can be shared (by default with 'edit' access) to other gmail users.

In [7]:
sheet.resource.share(user2)

In [8]:
sheet.share_delete(user2)

To share with 'view' access you pass an extra argument, `share_type='reader'`. You can also send a notification (by default there is no mail sent).

In [9]:
sheet.share(user2)
sheet.share_modify(user2,share_type="reader",send_notifications=True)

In [10]:
sheet.share_list()

[('n.lawrence@sheffield.ac.uk', 'reader'), ('lawrennd@gmail.com', 'owner')]

The spreadsheet's title can also be renamed.

In [11]:
sheet.title = 'New Title'

And the new title exhibited.

In [12]:
sheet.title

'New Title'

Or it can be deleted.

In [13]:
sheet.resource.delete()

### Loading an Existing Sheet

Now, for demonstration pruposes, here's the key of 'one we made earlier'. You can open an existing spreadsheet by providing the key as follows

In [14]:
resource = gl.drive.Resource(id='1zV1LT5_ITZuycRwv7iM2wLHH61PjMtKaPPIFYv7NdY8', drive=sheet.resource.drive)
sheet = gl.sheet.Sheet(resource=resource)

In [15]:
sheet

,one,two
,,
b,2,2
d,,4
a,1,1


In [16]:
sheet.embedurl

'https://docs.google.com/spreadsheets/d/e/1zV1LT5_ITZuycRwv7iM2wLHH61PjMtKaPPIFYv7NdY8/pubhtml?widget=true&amp;headers=false'

For this google doc, now it's created we've published it to the web. Quoting from [the developer's site](https://developers.google.com/google-apps/spreadsheets/)

"Publishing a spreadsheet to the web can only be done from the Google sheet's user interface. To start publishing a spreadsheet to the web, select `File > Publish` to the web from the sheet's user interface, and then click the Start Publishing button."

Once the spreadsheet is published you can have people viewing it publicly as follows.

In [34]:
sheet.resource.drive.service.files().get(fileId=sheet.id, fields="mimeType").execute()["mimeType"]

'application/vnd.google-apps.spreadsheet'

In [30]:
sheet.resource.get_name()

'Cat'

In [18]:
sheet.resource.drive.service.files().get(fileId=sheet.id, fields="*").execute()

{'kind': 'drive#file',
 'id': '1zV1LT5_ITZuycRwv7iM2wLHH61PjMtKaPPIFYv7NdY8',
 'name': 'Cat',
 'mimeType': 'application/vnd.google-apps.spreadsheet',
 'starred': False,
 'trashed': False,
 'explicitlyTrashed': False,
 'parents': ['0AL1UHKz3CH09Uk9PVA'],
 'spaces': ['drive'],
 'version': '113',
 'webViewLink': 'https://docs.google.com/spreadsheets/d/1zV1LT5_ITZuycRwv7iM2wLHH61PjMtKaPPIFYv7NdY8/edit?usp=drivesdk',
 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/vnd.google-apps.spreadsheet',
 'hasThumbnail': True,
 'thumbnailLink': 'https://docs.google.com/feeds/vt?gd=true&id=1zV1LT5_ITZuycRwv7iM2wLHH61PjMtKaPPIFYv7NdY8&v=15&s=AMedNnoAAAAAYdd7EH399EVNxvxUFUdKGne57FYqg-41&sz=s220',
 'thumbnailVersion': '15',
 'viewedByMe': True,
 'viewedByMeTime': '2022-01-06T18:06:26.215Z',
 'createdTime': '2022-01-05T22:28:39.145Z',
 'modifiedTime': '2022-01-06T21:28:16.486Z',
 'modifiedByMeTime': '2022-01-06T21:28:16.486Z',
 'modifiedByMe': True,
 'owners': [{'kind': 'dr

In [19]:
sheet.share([user1, user2])

Or you can modify the access rights they have to the document.

In [20]:
sheet.share_modify(user2, 'writer')

Or you can remove them from the sharing list completely.

In [21]:
sheet.share_delete(user1)

To see who the document is shared with you can use the `share_list()` method.

In [22]:
sheet.share_list()

[('n.lawrence@sheffield.ac.uk', 'writer'),
 ('Anyone with link', 'writer'),
 ('lawrennd@gmail.com', 'owner')]

In [23]:
%debug

### Displaying the Sheet

You can display the sheet in IPython, this action downloads the sheet and shows the associated `pandas` dataframe.

In [24]:
from IPython.display import display
display(sheet)


,one,two
,,
b,2,2
d,,4
a,1,1


Or if the spreadsheet is published you can set `sheet.published = True` and get a representation of the spreadsheet embedded in the notebook.

## Reading and Modifying the Google Sheet

To read the data frame back from the spreadsheet you simply use the `read` method.

In [25]:
df2 = sheet.read()
df2

,one,two
,,
b,2,2
d,,4
a,1,1


We can now update the spreadsheet, by modifying the data frame, and then requesting an update. The update command looks only for entries that have changed.

In [26]:
df2

,one,two
,,
b,2,2
d,,4
a,1,1


In [27]:
df2

,one,two
,,
b,2,2
d,,4
a,1,1


In [28]:
df2

,one,two
,,
b,2,2
d,,4
a,1,1


In [29]:
df2.at["a", "one"] = 2.3
sheet.update(df2)
sheet

NameError: name 'df_fal' is not defined

You should see the spreadsheet has changed. You can also delete rows.

In [ ]:
sheet.update(df2.drop("a"))
display(sheet)

Or add them in again. Here we update back to the original data frame (`df`). Note when you look at the google doc that the row is added at the end of the spreadsheet. This is so that any sorting of other rows in the spreadsheet is preserved.

In [ ]:
sheet.update(df)

Let's open the sheet again with a new interface.

In [ ]:
%debug

In [ ]:
sheet2 = gl.sheet.Sheet(resource=sheet.resource)
df4 = sheet2.read()
display(df4)

Note that because we deleted the row indexed by `"b"` and then updated the spreadsheet, the row of `b` was added back at the end of the spreadsheet.

In [ ]:
sheet2.update(df4.drop("c"))

In [ ]:
display(sheet2)

In [ ]:
sheet2._permission_id("lawrennd@gmail.com")